<a href="https://colab.research.google.com/github/kentokura/ox_2x2_retrograde_analysis/blob/main/ox2x2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ノードの３状態:
  - 未発見
  - 未訪問
  - 既訪問

処理:
- 初期状態"____"をunsolvedに追加する。 # unsolvedに積まれているノードは未訪問.
- unsolvedが空になるまで以下を行う # BFS開始
  1. unsolvedから先頭のノード(current_node)を抽出する。
  1. 抽出したノード(current_node)をunsolvedから削除する。
  1. ノード(current_node)から辿れる次のノード(next_nodes)全てを調べる。 # 探索
    1. もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
      1. そのノードを未訪問にする。 # unsolvedに追加
    1. そうではなく、発見済みならば
      1. 探索しない。
    1. next_node[previous_node]にcurrent_nodeを追加する。
    1. current_node[next_node]にnext_nodeを追加する。
  1. ノード（current_node）を既訪問にする。 # solvedに追加

1. solvedをプリントする。

# インプット

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# csvの読み込むためのモジュール
import pandas as pd
from pandas import DataFrame
import numpy as np


# 処理

In [ ]:
# state を入力すると、[次のstate]を出力する関数

def nextStates(state: str) -> list: 
  next_states = ["xxxxx"]
  if state == "____":
    next_states = ["o___"]
  elif state == "o___":
    next_states = ["ox__", "o__x"]
  elif state == "ox__":
    next_states = ["oxo_", "ox_o"]
  elif state == "o__x":
    next_states = ["oo_x"]
  elif state == "oxo_":
    next_states = ["oxox"]
  elif state == "ox_o":
    next_states = ["oxxo"]
  elif state == "oo_x":
    next_states = ["ooxx"]
  else:
    next_states = ["oooo"]
  return next_states

# stateを入力すると、正規化したstateを返す関数


In [ ]:
# unsolvedDf, solvedDfの初期化
print("===")
print("プログラム開始")
print("===")
print()
print("データを初期化します")
cols = ["PREVIOUS_STATES", "STATE", "NEXT_STATES"] #[前の状態list， 状態, 次の状態list]
df = pd.DataFrame(index=[], columns=cols)
df.set_index("STATE")
unsolvedDf = df
solvedDf = df
print("データを初期化しました")
print()

# 初期状態"____"をunsolvedに追加する。unsolvedに積まれているノードは未訪問.
print("===")
print("BFSの準備")
print("===")
print()
print("初期状態をセットします")
init_state = "____"
previous_state = ""
unsolvedDf = unsolvedDf.append(pd.Series([[previous_state], init_state, "unsolved"], index=df.columns, name=init_state))
print("初期状態をセットしました") # 確認
print("確認[UNSOLVED_DF]:") # 確認
print(unsolvedDf) # 確認
print() # 確認

# unsolvedが空になるまで以下を行う. BFS開始
print("===")
print("BFSを開始します")
print("===")
print()
for _ in range(20): # while len(unsolvedDf) > 0: # 開発のためにfor文にしている。
  # unsolvedDfから先頭のノードをpopする。
  if len(unsolvedDf) <= 0:
    break;
  current_node = unsolvedDf.iloc[0]  # 先頭のノード(current_node)を抽出。
  unsolvedDf.drop(unsolvedDf.index[0], inplace=True)  # 抽出したノードをunsolvedから削除。
  # 先頭のノード(current_node)から次のノード(next_nodes)を探索する。
  next_states = nextStates(current_node.STATE) # 次のノードの探索結果
  current_node.NEXT_STATES = next_states # current_nodeのNEXT_STATESに探索結果を反映
  # 探索した全ての状態について、以下を行う。
  print("unsolvedDfからpopされたノード'{}'の探索を行います".format(current_node.STATE))
  for next_state in next_states:
    # もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
    if (next_state not in unsolvedDf.STATE.values) and (next_state not in solvedDf.STATE.values):
      if next_state == current_node.STATE: # 次のノードが自身と同一
        continue;
      print("    探索結果: 未発見のノード'{}'です".format(next_state))
      # T)そのノードを未訪問にする。 # unsolvedに追加
      print("        unsolvedDfに追加します")
      print("        確認[unsolvedDf]")
      print(unsolvedDf)
      previous_state = [current_node.STATE]
      next_node = pd.Series([previous_state, next_state, "unsolved"], index=df.columns, name=next_state) # next_nodeの作成
      unsolvedDf = unsolvedDf.append(next_node)
      print("        unsolvedDfに追加しました")
      print("        確認[unsolvedDf]")
      print(unsolvedDf)
      print()
    else:  # F)そうではなく、発見済みならば
      print("    探索結果: 発見済みのノード'{}'です".format(next_state))
      #これを既に登録されていたノードのprevious_stateに追加する。
      previous_state = [current_node.STATE] 
      if next_state in unsolvedDf.STATE.values: # unsolvedDfに存在
        print("        これはunsolvedに存在しています")
        # unsolvedDf[unsolvedDf.STATE.values == next_state])にprevious_stateを追加する
        tmp = unsolvedDf.loc[next_state, "PREVIOUS_STATES"]
        tmp.append(previous_state[0])       
        unsolvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
      elif next_state in solvedDf.STATE.values:# solveDfに存在
        print("        これはsolvedに存在しています")
        # solvedDf[solvedDf.STATE.values == next_state])にprevious_stateを追加する
        tmp = solvedDf.loc[next_state, "PREVIOUS_STATES"]
        tmp.append(previous_state[0])       
        solvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
      else: # 何らかの理由で漏れた状態
        print("        エラー")
  # 現在のノード（current_node）をsolvedDfに追加する。solvedDfのノードは既訪問。 
  solvedDf = solvedDf.append(current_node)
print()
print("BFSが終了しました")
print()
# 結果確認
print("===")
print("結果確認")
print("===")
print()
print("確認[unsolvedDf]:")
print(unsolvedDf)
print()
print("確認[solvedDf]:")
print(solvedDf)
print()

===
プログラム開始
===

データを初期化します
データを初期化しました

===
BFSの準備
===

初期状態をセットします
初期状態をセットしました
確認[UNSOLVED_DF]:
     PREVIOUS_STATES STATE NEXT_STATES
____              []  ____    unsolved

===
BFSを開始します
===

unsolvedDfからpopされたノード'____'の探索を行います
    探索結果: 未発見のノード'o___'です
        unsolvedDfに追加します
        確認[unsolvedDf]
Empty DataFrame
Columns: [PREVIOUS_STATES, STATE, NEXT_STATES]
Index: []
        unsolvedDfに追加しました
        確認[unsolvedDf]
     PREVIOUS_STATES STATE NEXT_STATES
o___          [____]  o___    unsolved

unsolvedDfからpopされたノード'o___'の探索を行います
    探索結果: 未発見のノード'ox__'です
        unsolvedDfに追加します
        確認[unsolvedDf]
Empty DataFrame
Columns: [PREVIOUS_STATES, STATE, NEXT_STATES]
Index: []
        unsolvedDfに追加しました
        確認[unsolvedDf]
     PREVIOUS_STATES STATE NEXT_STATES
ox__          [o___]  ox__    unsolved

    探索結果: 未発見のノード'o__x'です
        unsolvedDfに追加します
        確認[unsolvedDf]
     PREVIOUS_STATES STATE NEXT_STATES
ox__          [o___]  ox__    unsolved
        unsolvedDfに追加しました
   

# 出力

In [ ]:
# solvedDfをox_outputという名前で書き出し
solvedDf.to_csv('/content/drive/My Drive/ox/workspace/ox_output.csv')

In [ ]:
# ox_outputの確認

solvedDf = pd.read_csv(
    "/content/drive/My Drive/ox/workspace/ox_output.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )

print(solvedDf)

               PREVIOUS_STATES STATE       NEXT_STATES
____                      ['']  ____          ['o___']
o___                  ['____']  o___  ['ox__', 'o__x']
ox__                  ['o___']  ox__  ['oxo_', 'ox_o']
o__x                  ['o___']  o__x          ['oo_x']
oxo_                  ['ox__']  oxo_          ['oxox']
ox_o                  ['ox__']  ox_o          ['oxxo']
oo_x                  ['o__x']  oo_x          ['ooxx']
oxox                  ['oxo_']  oxox          ['oooo']
oxxo                  ['ox_o']  oxxo          ['oooo']
ooxx                  ['oo_x']  ooxx          ['oooo']
oooo  ['oxox', 'oxxo', 'ooxx']  oooo          ['oooo']


計画

- [x] 入力、処理、出力の雛形作成
- [x] ２ｘ２のoxゲームを実装

## 2x2

- 探索しよう。
初期状態とルールか。
- 状態をぶちこむと、次の状態を返す関数。
- 次の状態からｄｆを適切に変更。

In [ ]:
# nodeの勝ち負け判定に使う。
# csvの読み込み
df = pd.read_csv(
    "/content/drive/My Drive/ox/workspace/ox.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(df)
# print(df.query("id==0").to_numpy(copy=True).flatten())

    front_id  next_id    state
id                            
0        NaN        1  _,_,_,_
